# Mega-Sena

- **Aluno**: Álvaro Ferreira Pires de Paiva
- **Matrícula**: 2016039162

**Sumário**
- [Base de dados](#database)
- [Preparando o ambiente](#env_treatment)
  - Prêmio máximo
  - Rótulo
- [Análises](#analysis)
  - Dicas

## <a id="database">Base de dados</a>

Nessa avaliação será utilizado a base de dados referente aos concursos da Mega-Sena. 

A **Mega-Sena** tem sido a maior modalidade lotérica do Brasil, sendo uma entre as dez modalidades atuais das loterias da Caixa (Caixa Econômica Federal), com sorteios ordinários duas vezes por semana, além da Mega-Sena da Virada.

Para ganhar o **prêmio máximo da Mega-Sena** — **a sena** — , é necessário acertar a sena, o que significa obter coincidência entre **seis dos números** apostados e os seis números sorteados, de um total de sessenta dezenas (**de 01 a 60**), independentemente da ordem da aposta ou da ordem do sorteio. 

O concurso prevê também a chance de se ganhar parte do prêmio máximo, pelo acerto da **quina** (apenas cinco dos números sorteados), ou da **quadra** (apenas quatro dos números sorteados), com prêmios significativamente menores que aquele que seria pago na ocorrência do acerto da sena, o da quina maior que o da quadra. 

A base de dados está em formato de planilha em excel.

## <a id="env_treatment">Preparando o ambiente</a>
Aqui iremos realizar as importações de bibliotecas necessárias e o armazenamento dos dados para podermos analisarmos logo em seguida.
Dependencies
```
!conda install bokeh
```

In [1]:
# Imports
import numpy as np
import pandas as pd
from bokeh.models import HoverTool
from bokeh.plotting import figure, output_notebook, show

In [2]:
# Leitura dos dados
excel_file = 'basedados.xlsx'
excel = pd.ExcelFile(excel_file)
data = excel.parse(0)

### Prêmio máximo
Para cada concurso da Mega-Sena, a maior premiação está relacionada com o acerto das 6 dezenas, e o seu valor é dada pela seguinte relação:

$$Prêmio = Ganhadores\_Sena	\times Rateio\_Sena$$

In [3]:
data['premio_maximo'] = data.Ganhadores_Sena * data.Rateio_Sena

### Rótulo
Criaremos uma nova coluna chamada "Rótulo" baseada na faixa de valores da coluna "prêmio_máximo", como apresentado na seguinte tabela:

| Intervalo [Milhões de Reais] | Rótulo |
|------------------------------|--------|
| 0 - 1                        | I      |
| 1 - 5                        | II     |
| 5 - 10                       | III    |
| 10 - 20                      | IV     |
| 20 - 40                      | V      |
| 40 - 80                      | VI     |
| 80 - 100                     | VII    |
| 100 - 200                    | VIII   |
| 200 - 300                    | IX     |
| 300 -                        | X      |

In [4]:
data['rotulo'] = pd.cut(data.premio_maximo,
                        bins=[0,1e6,5e6,1e7,2e7,4e7,8e7,1e8,2e8,3e8,1e9],
                        labels=["I","II","III","IV","V","VI","VII","VIII","IX","X"],
                        include_lowest=True)

## <a id="analysis">Análises</a>
Nessa seção, iremos realizar algumas análises nos dados que temos em mãos.

### Dicas
Pela Mega-Sena possuir prêmios de valores bastantes elevados, é normal encontrar sites com dicas para o jogador ter mais chances de ganhar. Sendo assim, iremos análisar 8 dicas encontradas em alguns sites e verificar a sua veracidade. As dicas são:

1. Os números com final 9 ou 0 saem pouco, por isso, é bom evitar escolher várias números com essa sequencia.
2. As dezenas 01, 02, 03, 11, 22, 44, 55, 48 e 57 saem pouco.
3. Não jogue números seguidos.
4. Não jogue em números que estejam na mesma linha vertical.
5. Divida a cartela em quatro quadrantes e distribua seu jogo entre eles.
6. Jogue sempre a mesma quantidade de dezenas pares e ímpares. Na Mega-Sena, 81% dos sorteios têm o seguinte esquema: 3 números pares e 3 ímpares ou 4 pares e 2 ímpares, ou vice-versa.
7. Prefira apenas um cartão com mais de seis dezenas do que vários cartões de seis dezenas. A regra é simples: quanto mais números você apostar, maiores são as chances de acertar um deles.
8. Um jogo em que se preenche 12 dezenas utilizando essas regras teria mais chances de êxito. Logo, o bolão é a melhor chance que se tem de ganhar.

Algumas variáveis iremos utilizar na validação de mais de uma dica, sendo elas:

In [5]:
# Colunas que iremos trabalhar
cols_dezenas = ['Dezena1', 'Dezena2', 'Dezena3', 'Dezena4', 'Dezena5', 'Dezena6']

# Dataframe filtrado pelas colunas que queremos
data_cols = data[cols_dezenas]

# Dataframe com os últimos dígitos de cada número de cada jogo
dt_last_digit = data_cols.apply(lambda x: x%10)

# Dezenas da mega-sena
dezenas = [i for i in range(1, 61)]

#### 1. Os números com final 9 ou 0 saem pouco

In [6]:
# Vetor com índices e 0 a 9 que usaremos para contar
count_last_digit = [0 for i in range(0, 10)]

# Iremos percorrer cada celula do dataframe e verificar sua terminação
for np_array in data_cols.values:
    for number in np_array:
        count_last_digit[ number % 10 ] += 1

In [7]:
# Total de números contados
last_digit_total = sum(count_last_digit)

# Ordenamos a lista pela quantidade de ocorrências de cada dígito (do menor para o maior)
last_digit_sort = sorted(enumerate(count_last_digit), key=lambda x:x[1])
# Invertemos a ordem para facilitar a visualização
last_digit_sort.reverse()

# Imprimimos a porcentagem de cada um
for index, value in last_digit_sort:
    print("{} - {}".format(index, (value*100/last_digit_total)))

3 - 10.576131687242798
4 - 10.444444444444445
2 - 10.148148148148149
0 - 10.08230452674897
7 - 9.958847736625515
8 - 9.893004115226338
1 - 9.868312757201647
6 - 9.76954732510288
5 - 9.670781893004115
9 - 9.588477366255145


In [8]:
# Gráfico para melhor visualização
dica1 = figure(plot_width=500, plot_height=500)
dica1.vbar(x=[i for i,v in last_digit_sort],
       width=0.5,
       bottom=0,
       top=[v for i,v in last_digit_sort])

dica1.add_tools(HoverTool(tooltips = [
    ("Dígito", "@x"),
    ("Ocorrências", "@top"),
]))

output_notebook()
show(dica1)

Loading BokehJS ...

Através das informações apresentadas logo acima, podemos constatar que:
- Números com último dígito sendo **0** possuem a quarta maior ocorrência nos sorteios até agora;
- Realmente os números com último dígito sendo **9** são os que menos saem nos sorteios. 

Sendo assim, a dica é verdade quando se trata apenas do dígito 9.

#### 2. As dezenas 01, 02, 03, 11, 22, 44, 55, 48 e 57 saem pouco

In [9]:
# Vetor com dígitos de 1 a 60
count_digit = [0 for i in range(0, 60)]

# Iremos percorrer cada celula do dataframe
for np_array in data_cols.values:
    for number in np_array:
        count_digit[ number-1 ] += 1

In [10]:
# Ordenamos a lista pela quantidade de ocorrências de cada dígito (do menor para o maior)
digit_sort = sorted(enumerate(count_digit), key=lambda x:x[1])
# Invertemos a ordem para facilitar a visualização
digit_sort.reverse()

In [11]:
# Dezenas informadas na dica
find_digits = [1, 2, 3, 11, 22, 44, 55, 48, 57]

# Mostramos o lugar no ranking de ocorrências 
for digit, value in digit_sort:
    if digit in find_digits:
        print("Dezena: {} | Lugar no ranking de ocorrências: {}".format(digit, digit_sort.index((digit, value))))

Dezena: 22 | Lugar no ranking de ocorrências: 3
Dezena: 3 | Lugar no ranking de ocorrências: 4
Dezena: 1 | Lugar no ranking de ocorrências: 21
Dezena: 55 | Lugar no ranking de ocorrências: 23
Dezena: 44 | Lugar no ranking de ocorrências: 32
Dezena: 48 | Lugar no ranking de ocorrências: 34
Dezena: 11 | Lugar no ranking de ocorrências: 37
Dezena: 57 | Lugar no ranking de ocorrências: 39
Dezena: 2 | Lugar no ranking de ocorrências: 46


Através das informações apresentadas acima, podemos ver que a dica 2 é falsa.

#### 3. Não jogue números seguidos

In [12]:
ocor_seguidos = 0
ocor_not_seguidos = 0

# Percorremos cada jogo da mega
for np_array in data_cols.values:
    # Ordenamos o array
    np_array = np.sort(np_array)
    
    # Contamos a quantidade de ocorrências de números seguidos
    for index, number in enumerate(np_array[1:]):
        # Verificamos se os números formam uma sequência
        if np_array[index] == (np_array[index+1]-1):
            ocor_seguidos += 1
        else:
            ocor_not_seguidos += 1
            
print("Ocorrências de números seguidos: {}".format(ocor_seguidos))
print("Ocorrências de números que não formam uma sequência: {}".format(ocor_not_seguidos))

Ocorrências de números seguidos: 994
Ocorrências de números que não formam uma sequência: 9131


Podemos ver que a quantidade de vezes que dois números formaram uma sequência foi ínfima se comparado a quantidade de números jogados até agora. Portanto, a dica é verdadeira.

#### 4. Não jogue em números que estejam na mesma linha vertical

In [20]:
# Contador de ocorrências de dezenas de mesma linha vertical e de diferentes linhas verticais, respectivamente
dv_ocor = 0
not_dv_ocor = 0

# Percorremos as linhas
for np_array in data_cols.values:
    # Usaremos essa variável para pegar as linhas verticais presentes no jogo sorteado
    dv_digit = []

    # Percorremos os números
    for number in np_array:
        # Último dígito da dezena
        last_digit = number%10
        # Se o dígito não foi salvo anteriormente, ele salva
        if not (last_digit in dv_digit):
            dv_digit.append(last_digit)
    
    # Salvamos a quantidade de linhas verticais do jogo sorteado 
    dv_ocor += (6 - len(dv_digit))
    not_dv_ocor += len(dv_digit)
    
por_dv_ocor = (dv_ocor*100)/(dv_ocor+not_dv_ocor)
    
print("Porcentagem de números que pertencem a mesma linha vertical: {}%".format(por_dv_ocor))

Porcentagem de números que pertencem a mesma linha vertical: 19.613168724279834%


Como podemos ver, pouco mais de 19.5% dos números sorteados na mega-sena pertenciam a mesma linha vertical de outra dezena no mesmo jogo. Sendo assim, é correto afirmar que devemos evitar dezenas que pertençam a mesma linha vertical.

#### 5. Divida a cartela em quatro quadrantes e distribua seu jogo entre eles

In [52]:
# Separamos as dezenas em grupos de 5 dezenas
dezenas_q = [dezenas[i:(i+5)] for i in range(0, 61, 5)]
del(dezenas_q[-1])

# Variável que usaremos para acoplar as dezenas de cada quadrante
dezenas_quadrantes = [[] for i in range(0, 4)]

# Organizar os quadrantes
for i in range(0, 12, 2):
    if i < 6:
        dezenas_quadrantes[0] = dezenas_quadrantes[0] + dezenas_q[i]
        dezenas_quadrantes[1] = dezenas_quadrantes[1] + dezenas_q[i+1]
    else:
        dezenas_quadrantes[2] = dezenas_quadrantes[2] + dezenas_q[i]
        dezenas_quadrantes[3] = dezenas_quadrantes[3] + dezenas_q[i+1]
        
dezenas_quadrantes

[[1, 2, 3, 4, 5, 11, 12, 13, 14, 15, 21, 22, 23, 24, 25],
 [6, 7, 8, 9, 10, 16, 17, 18, 19, 20, 26, 27, 28, 29, 30],
 [31, 32, 33, 34, 35, 41, 42, 43, 44, 45, 51, 52, 53, 54, 55],
 [36, 37, 38, 39, 40, 46, 47, 48, 49, 50, 56, 57, 58, 59, 60]]

**Quadrantes**:
<table>
    <tbody>
        <tr>
            <td>dezenas_quadrantes[0]</td>
            <td>dezenas_quadrantes[1]</td>
        </tr>
        <tr>
            <td>dezenas_quadrantes[2]</td>
            <td>dezenas_quadrantes[3]</td>
        </tr>
    </tbody>
</table>

In [58]:
quadrantes_count = [0, 0, 0, 0]
quad_count_yn = [0, 0]

# Percorremos as linhas
for np_array in data_cols.values:
    # Usaremos essa variável para pegar os quadrantes envolvidos
    quad_envolvidos = []

    # Percorremos os números
    for number in np_array:
        
        # Percorremos os quadrantes
        for index, quadrante in enumerate(dezenas_quadrantes):
            # Verificamos em qual quadrante o número se encontra
            if number in quadrante:
                quadrantes_count[index] += 1
                # Verifica se o quadrante já havia saído antes no mesmo jogo
                if not (index in quad_envolvidos):
                    quad_envolvidos.append(index)
                break
                
    
    if len(quad_envolvidos) < 4:
        quad_count_yn[0] += 1
    else:
        quad_count_yn[1] += 1
        
quad_count_yn

[1140, 885]

In [57]:
# Imprimimos

12150

In [59]:
sum(quad_count_yn)

2025